# 00 - Data exploration

Open and inspect raw and merged datasets to understand schemas and missingness.

#  
### Objective
Explore and understand the Big Data Bowl 2026 dataset (input, output, supplementary).
Identify which fields are available, what’s missing, and what must be reconstructed.


In [7]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [8]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

nfl_big_data_bowl_2026_analytics = kagglehub.competition_download('nfl-big-data-bowl-2026-analytics')

print('Data source import complete.')

100%|██████████| 103M/103M [00:00<00:00, 137MB/s] 

Extracting files...


Data source import complete.


In [16]:
print(nfl_big_data_bowl_2026_analytics)

/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics


In [17]:
!ls /root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics

114239_nfl_competition_files_published_analytics_final


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [13]:
import pandas as pd
import os

# Define the path to the data directory
data_dir = '/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/train'

# Construct the full path to the input data for week 1
week1_input_path = os.path.join(data_dir, 'input_2023_w01.csv')

# Load the week 1 input data into a pandas DataFrame
week1_input_df = pd.read_csv(week1_input_path)

# Display the first few rows of the DataFrame
display(week1_input_df.sample(20))

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
44231,2023091001,2759,False,45052,3,left,83,Noah Brown,6-2,225,...,Other Route Runner,83.88,20.72,0.87,3.63,276.54,343.36,11,69.089996,36.099998
223940,2023091011,1759,True,54548,1,right,67,Nakobe Dean,6-0,225,...,Defensive Coverage,71.50,25.90,0.27,1.03,259.42,254.80,19,86.930000,4.520000
45161,2023091001,3216,True,42393,4,left,58,Ronald Darby,5-11,193,...,Defensive Coverage,49.52,43.20,0.05,0.63,335.52,49.64,13,41.230000,38.610001
177687,2023091008,4452,False,44990,5,left,85,Trent Taylor,5-8,181,...,Other Route Runner,86.27,18.86,1.61,4.75,277.76,276.32,8,82.709999,18.000000
2766,2023090700,713,False,54716,11,left,83,Isiah Pacheco,5-11,215,...,Other Route Runner,87.70,26.55,2.38,5.39,35.19,287.62,8,79.910004,8.970000
46326,2023091001,3302,True,41238,21,right,33,Odell Beckham,5-11,198,...,Targeted Receiver,39.76,43.65,5.79,5.27,61.58,28.71,12,39.560001,50.410000
18347,2023090700,3324,False,55920,12,left,31,Rashee Rice,6-2,203,...,Other Route Runner,30.17,33.08,5.85,3.93,243.48,247.74,9,33.439999,34.369999
263817,2023091013,2213,False,43424,16,right,53,Dak Prescott,6-2,238,...,Passer,47.37,29.57,0.39,1.84,237.87,152.45,7,58.130001,21.760000
26307,2023091000,1429,True,53469,13,right,106,Richie Grant,6-0,194,...,Defensive Coverage,109.12,16.75,1.32,5.24,179.97,217.10,9,106.809998,9.540000
67975,2023091003,263,False,55868,16,left,102,Anthony Richardson,6-4,232,...,Passer,108.90,23.20,2.05,1.07,105.88,296.39,7,100.779999,13.460000


In [18]:
import pandas as pd
import os

# Define the path to the data directory
data_dir = '/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final'

# Construct the full path to the output data for week 1
week1_output_path = os.path.join(data_dir, 'train', 'output_2023_w01.csv')

# Load the week 1 output data into a pandas DataFrame
week1_output_df = pd.read_csv(week1_output_path)

# Display the first few rows of the DataFrame
print("First 5 rows of output_2023_w01.csv:")
display(week1_output_df.head())

First 5 rows of output_2023_w01.csv:


,game_id,play_id,nfl_id,frame_id,x,y
0,2023090700,101,46137,1,56.22,17.28
1,2023090700,101,46137,2,56.63,16.88
2,2023090700,101,46137,3,57.06,16.46
3,2023090700,101,46137,4,57.48,16.02
4,2023090700,101,46137,5,57.91,15.56


In [19]:
import pandas as pd
import os

# Define the path to the data directory
data_dir = '/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final'

# Construct the full path to the supplementary data
supplementary_data_path = os.path.join(data_dir, 'supplementary_data.csv')

# Load the supplementary data into a pandas DataFrame
supplementary_df = pd.read_csv(supplementary_data_path)

# Display the first few rows of the DataFrame
print("First 5 rows of supplementary_data.csv:")
display(supplementary_df.head())

First 5 rows of supplementary_data.csv:


/tmp/ipython-input-3401357624.py:11: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplementary_df = pd.read_csv(supplementary_data_path)


,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,team_coverage_type,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added
0,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,3461,(10:46) (Shotgun) J.Goff pass deep left to J.R...,4,...,COVER_2_ZONE,NaN,18,18,-0.664416,2.945847,0.834296,0.165704,-0.081149,0.081149
1,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,461,(7:30) J.Goff pass short right to J.Reynolds t...,1,...,COVER_6_ZONE,NaN,21,21,1.926131,1.345633,0.544618,0.455382,-0.029415,0.029415
2,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1940,(:09) (Shotgun) J.Goff pass incomplete deep ri...,2,...,COVER_2_ZONE,NaN,0,0,0.281891,-0.081964,0.771994,0.228006,0.000791,-0.000791
3,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1711,"(:45) (No Huddle, Shotgun) P.Mahomes pass deep...",2,...,COVER_2_ZONE,NaN,26,26,3.452352,2.342947,0.663187,0.336813,0.041843,-0.041843
4,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1588,(1:54) (Shotgun) P.Mahomes pass incomplete dee...,2,...,COVER_4_ZONE,NaN,0,0,1.921525,-0.324035,0.615035,0.384965,0.000061,-0.000061


Nice — here’s a clean, action-oriented **step-by-step checklist** for `01_data_exploration.ipynb`.
Each step contains: **what to do**, **why it matters**, and **what to produce/check** so you’ll always know *what* and *why* you did it.

Copy this into your notebook as headings and run each step one-by-one.

---

## Step 0 — Notebook header

**Aim:** declare purpose and keep the notebook self-documenting.
**Do:** add a top markdown cell:

* Title: `01_data_exploration.ipynb`
* Objective: “Inspect input/output/supplementary; confirm throw-window logic; list fields to reconstruct; produce Data Schema & Time Logic Summary.”
  **Produce:** short header so anyone opening the notebook knows the goal.

---

## Step 1 — Imports & file paths

**Aim:** make environment reproducible and files easy to load.
**Do:** import pandas, numpy, matplotlib, seaborn (optional), pathlib. Set data paths for `data/raw/`.
**Check:** notebook runs without import errors.
**Produce:** a small code cell showing file paths.

---

## Step 2 — Quick file load & shape check

**Aim:** confirm files load and see basic row/col counts.
**Do:** `pd.read_csv()` each of `input_*.csv`, `output_*.csv`, `supplementary.csv` for at least one week file. Print `.shape` and `.columns`.
**Why:** ensures files exist and columns match expectations; reveals immediate size/memory considerations.
**Produce / Check:** prints like:

* `Input shape: (n_rows, n_cols)`
* `Output shape: ...`
* `Supplementary shape: ...`

---

## Step 3 — Column inventory per file

**Aim:** list what fields exist where (quick data dictionary).
**Do:** for each df run `df.columns.tolist()` and `df.dtypes`. Note fields present only in input (e.g., `s, a, o, dir, ball_land_x/y`), and fields present only in output (`frame_id`, `x,y` after throw).
**Why:** identifies exactly what must be reconstructed or merged.
**Produce:** a tiny markdown table showing `Field | Input | Output | Supplementary`.

---

## Step 4 — Missingness & types

**Aim:** detect nulls, weird types, or garbage values early.
**Do:** `df.isna().sum()` and `df.describe()` for numeric ranges (especially `x`, `y`). Look for NaNs in `ball_land_x/y`, `num_frames_output`, or `frame_id`.
**Why:** missing values affect merging, reconstruction, and metric validity.
**Produce / Check:** list of columns with >0% missing; note whether `x,y` have invalid ranges (<0 or >field dims).

---

## Step 5 — Uniqueness & key-structure validation

**Aim:** verify identity integrity fundamentals.
**Do:** run checks:

* `input_df.duplicated(['game_id','play_id','nfl_id']).any()`
* `output_df.duplicated(['game_id','play_id','nfl_id','frame_id']).any()`
* `supp_df.duplicated(['game_id','play_id']).any()`
  Also check `input`’s `num_frames_output` vs `output` max `frame_id` per (game_id,play_id,nfl_id).
  **Why:** prevents silent ID mixing later.
  **Produce / Check:** assert tests or a small report listing any violations (if violations exist, log them).

---

## Step 6 — Frame rate & temporal consistency

**Aim:** confirm frame rate (frames per second) and per-play frame continuity.
**Do:** for several plays compute `max(frame_id)` and infer duration as `max_frame / assumed_frame_rate` (assume 10Hz initially). Check if `frame_id` is sequential 1..N.
**Why:** smoothing and derivative formulas depend on correct Δt.
**Produce / Check:** list of sample plays with `max_frame`, and note if any non-1 start or gaps exist.

---

## Step 7 — Cross-file play matching

**Aim:** ensure plays in input map to output and supplementary.
**Do:** compute counts per play and compare:

* Unique `(game_id,play_id)` in input vs supplementary vs output.
  Report plays present in input but missing in output, etc.
  **Why:** some plays may be filtered out (penalties, scrambles); know which to drop.
  **Produce / Check:** small Venn or counts and list of mismatched play ids.

---

## Step 8 — QuickVisualization: single-play sanity check

**Aim:** visually confirm coordinate orientation and consistency.
**Do:** pick 2–3 sample plays (one deep pass, one short, one contested). Plot `x,y` before throw (input) and after throw (output) on same axes. Color by `frame_id` (use alpha).
**Why:** confirms coordinate system (0–120 x axis, 0–53.3 y) and play_direction orientation.
**Produce / Check:** 2 plots per play: pre-throw scatter + post-throw trace. Verify shapes look like football plays, not noisy clouds.

---

## Step 9 — Identify target / receiver & defender sets

**Aim:** make sure you can identify targeted receiver and defender pool per play.
**Do:** from input look at `player_role` and `player_side` to find `Targeted Receiver` and `Defensive Coverage`. For each targeted receiver, list defenders present (their `nfl_id`s) in output frames.
**Why:** SG and CCI need nearest defenders and defender sets.
**Produce / Check:** sample mapping for a play like `target_nfl_id: [def1, def2,...]`.

---

## Step 10 — List fields to reconstruct and how

**Aim:** finalize the reconstruction checklist so future work is clear.
**Do:** create a markdown table of fields you must derive from `x,y` in output:

* `s` = speed = sqrt((dx/dt)^2 + (dy/dt)^2)
* `vpx, vpy` = velocity components = dx/dt, dy/dt
* `a` = acceleration = dv/dt
* `dir` or heading = atan2(dy, dx)
* `closing radial velocity` = projection of velocity toward ball_land point
  **Why:** explicit formulas let you prototype and test consistently.
  **Produce:** the table with formula lines and note to compute on **smoothed** coordinates (not raw).

---

## Step 11 — Small statistical checks for distributions

**Aim:** get baseline stats to detect outliers and guide smoothing thresholds.
**Do:** compute distribution summaries for a sample of reconstructed-like values (or for `s`/`a` if present in input file): mean, median, max. Note plausible physical bounds (max speed ~ 11–12 yd/s).
**Why:** helps choose smoothing parameters and speed/accel caps.
**Produce / Check:** histograms for speed & acceleration; flag any extreme outliers for inspection.

---

## Step 12 — Produce the “Data Schema & Time Logic Summary” (deliverable)

**Aim:** create the one-page reference describing what you have and the reconstruction needs.
**Do:** write a single markdown cell with:

* Dataset files used
* Key fields present in each file
* The throw-window definition (output frames = ball-in-air)
* Fields to reconstruct and formulas (from Step 10)
* Integrity checks performed and their results (Step 5 & 7)
  **Why:** this is your official checkpoint before reconstruction — include in `reports/00_data_availability.md`.
  **Produce:** one-page summary saved to repo.

---

## Step 13 — Save a small merged sample for next notebook

**Aim:** prepare a tiny working file for reconstruction prototyping.
**Do:** merge one or two fully validated plays (on keys `game_id,play_id,nfl_id`) into a small parquet `data/interim/merged_sample.parquet` containing both input columns (player attributes + ball_land) and output frames for those plays.
**Why:** speeds up development and avoids repeatedly reloading massive files.
**Produce:** `merged_sample.parquet` (2–5 plays).

---

## Step 14 — Quick checklist & next actions

**Aim:** finalize and move to reconstruction with clarity.
**Do:** confirm:

* All uniqueness checks passed or logged.
* You have sample plays and `merged_sample.parquet`.
* Data Schema & Time Logic Summary saved.
* Notebook saved & committed to GitHub.
  **Why:** ensures reproducibility and a clear handoff to `02_data_reconstruction.ipynb`.
  **Produce:** short todo list for next notebook: smoothing method(s) to test, validation plays, config constants (frame_rate, smoothing windows).

---

### Small code hints (copy-paste friendly)

* Uniqueness check example:

```python
assert not output_df.duplicated(['game_id','play_id','nfl_id','frame_id']).any()
```

* Max frame vs num_frames_output:

```python
check = merged.groupby(['game_id','play_id','nfl_id']).agg(max_frame=('frame_id','max'), expected=('num_frames_output','first'))
(check['max_frame']==check['expected']).value_counts()
```

* Simple velocity estimate (for prototyping):

```python
df = df.sort_values('frame_id')
df['dx'] = df['x'].diff()
df['dy'] = df['y'].diff()
df['s_raw'] = np.sqrt(df['dx']**2 + df['dy']**2) * frame_rate  # frame_rate e.g. 10
```

---

If you follow these steps, at the end of this notebook you will have:

* A crystal-clear understanding of what data exists and what must be reconstructed,
* Validation that IDs and frames align,
* A saved small merged sample for fast prototyping,
* The one-page Data Schema & Time Logic Summary that documents the decisions.

Ready to start? If you want, I can now produce a ready-to-copy cell template for each step (code + markdown) so you can paste directly into `01_data_exploration.ipynb`.
